### Installation

In [1]:
# Do this only in Colab notebooks! Otherwise use pip install unsloth
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
models_name = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Llama-3.2-3B-unsloth-bnb-4bit",
    "unsloth/Llama-3.2-1B-bnb-4bit",
    # not pre quantized models
    "unsloth/Meta-Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.2-1B",
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
] # More models at https://huggingface.co/unsloth



model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = models_name[6],
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

**LoRA (Low-Rank Adaptation)** is a technique used to fine-tune large language models efficiently. Instead of updating all model weights, it adds small trainable matrices to certain layers, drastically reducing the number of parameters to train. This makes fine-tuning faster, cheaper, and requires less data.

In **LoRA**, the added matrix is built using a **low-rank decomposition**:

* Instead of learning a full weight update matrix **ΔW** (which is large), LoRA approximates it as:
  **ΔW ≈ A × B**,
  where:

  * **A** has shape *(original\_dim, r)*
  * **B** has shape *(r, input\_dim)*
  * **r** is a small number (the **rank**), usually much smaller than the original dimensions

This means:

* Only **A** and **B** are trained (not the full matrix),
* The product **A × B** is added to the frozen original weights during training,
* This approach reduces the number of trainable parameters from **O(n²)** to **O(n × r)**.

This low-rank trick makes LoRA both efficient and effective.

The **`lora_alpha`** parameter is a **scaling factor** used in LoRA to control the impact of the low-rank update.

After computing the low-rank approximation $\Delta W = A \times B$, the scaled update is:

$$
\Delta W_{\text{scaled}} = \frac{\alpha}{r} \cdot (A \times B)
$$

Where:

* **$\alpha$** is **`lora_alpha`**
* **$r$** is the **rank** (size of the bottleneck)
* $\frac{\alpha}{r}$ ensures the update has the right magnitude

### Purpose:

* It balances how much the LoRA module influences the model during training.
* Higher `lora_alpha` → stronger effect of the LoRA update.
* Lower `lora_alpha` → weaker effect, more conservative adaptation.

This allows fine control without retraining everything.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!



In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.7 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [4]:
!git clone https://huggingface.co/datasets/crux82/squad_it

Cloning into 'squad_it'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 59 (delta 0), reused 0 (delta 0), pack-reused 53 (from 1)
Unpacking objects: 100% (59/59), 15.14 KiB | 149.00 KiB/s, done.


<a name="Data"></a>
### Data Prep
We now use the SQUAD-it dataset from [Roma Tor Vergata](https://huggingface.co/datasets/crux82/squad_it), which is s derived from the SQuAD dataset and it is obtained through semi-automatic translation of the SQuAD dataset into Italian. You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

We now use the `Llama-3.1` format for conversation style finetunes.
We have to convert the dataset to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`.
Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template.

In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
  questions = examples["question"]
  contexts  = examples["context"]
  answers   = [e["text"][0] for e in examples["answers"]]
  texts = []
  for questions, contexts, answers in zip(questions, contexts, answers):
    messages = [
        {"role": "system", "content": "Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato."},
        {"role": "user", "content": "Dato il seguente contesto: {}\nRispondi alla domanda: {}".format(contexts, questions)},
        {"role": "assistant", "content": answers},
    ]
    texts.append(tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False))
  return { "text" : texts, }

pass

from datasets import load_dataset
dataset = load_dataset("./squad_it/data", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/54159 [00:00<?, ? examples/s]

In [6]:
print(dataset[0]["text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.<|eot_id|><|start_header_id|>user<|end_header_id|>

Dato il seguente contesto: Il terremoto del Sichuan del 2008 o il terremoto del Gran Sichuan, misurato a 8.0 Ms e 7.9 Mw, e si è verificato alle 02:28:01 PM China Standard Time all' epicentro (06:28:01 UTC) il 12 maggio nella provincia del Sichuan, ha ucciso 69.197 persone e lasciato 18.222 dispersi.
Rispondi alla domanda: In quale anno si è verificato il terremoto nel Sichuan?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

2008<|eot_id|>


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 100 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [7]:
import xformers
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 8,
        warmup_steps = 10,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/54159 [00:00<?, ? examples/s]

In [8]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.457 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 54,159 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

398.0883 seconds used for training.
6.63 minutes used for training.
Peak reserved memory = 2.412 GB.
Peak reserved memory for training = 0.955 GB.
Peak reserved memory % of max memory = 16.363 %.
Peak reserved memory for training % of max memory = 6.479 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
messages = [
        {"role": "system", "content": "Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato."},
        {"role": "user", "content": "Dato il seguente contesto: La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 dollari al barile a quasi 12 dollari a livello mondiale; i prezzi americani erano notevolmente più elevati. L'embargo ha causato una crisi petrolifera, o \"shock\", con molti effetti a breve e lungo termine sulla politica globale e sull' economia globale. Più tardi fu chiamato il \"primo shock petrolifero\", seguito dalla crisi petrolifera del 1979, definita il \"secondo shock petrolifero\".\nRispondi alla domanda: Quando è iniziata la crisi petrolifera del 1973?"},
    ]

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nDi seguito c\'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nDato il seguente contesto: La crisi petrolifera del 1973 iniziò nell\'ottobre 1973 quando i membri dell\'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell\'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell\'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 dollari al barile a quasi 12 dollari a livello mondiale; i prezzi americani erano notevolmente più elevati. L\'embargo ha causato una crisi petrolifera, o "shock", con molti effetti a breve e lungo termine sulla politica globale e sull\' economia globale. Più tardi fu chiamato il "primo shock petrolifero", seguito dalla

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
messages = [
        {"role": "system", "content": "Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato."},
        {"role": "user", "content": "Dato il seguente contesto: La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 dollari al barile a quasi 12 dollari a livello mondiale; i prezzi americani erano notevolmente più elevati. L'embargo ha causato una crisi petrolifera, o \"shock\", con molti effetti a breve e lungo termine sulla politica globale e sull' economia globale. Più tardi fu chiamato il \"primo shock petrolifero\", seguito dalla crisi petrolifera del 1979, definita il \"secondo shock petrolifero\".\nRispondi alla domanda: Quando è iniziata la crisi petrolifera del 1973?"},
    ]

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.<|eot_id|><|start_header_id|>user<|end_header_id|>

Dato il seguente contesto: La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 dollari al barile a quasi 12 dollari a livello mondiale; i prezzi americani erano notevolmente più elevati. L'embargo ha causato una crisi petrolifera, o "shock", con molti effetti a breve e lungo termine sulla politica globale e sull' economia globale. Più tardi fu chiamato il "primo shock petrolifero", seguito dalla crisi petrolife

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model.

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

# for saving the merged model (model + adapter)
#model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit",)
#model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_4bit",)

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
        {"role": "system", "content": "Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato."},
        {"role": "user", "content": "Dato il seguente contesto: La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Siria) proclamarono un embargo petrolifero. Alla fine dell'embargo, nel marzo 1974, il prezzo del petrolio era salito da 3 dollari al barile a quasi 12 dollari a livello mondiale; i prezzi americani erano notevolmente più elevati. L'embargo ha causato una crisi petrolifera, o \"shock\", con molti effetti a breve e lungo termine sulla politica globale e sull' economia globale. Più tardi fu chiamato il \"primo shock petrolifero\", seguito dalla crisi petrolifera del 1979, definita il \"secondo shock petrolifero\".\nRispondi alla domanda: Quando è iniziata la crisi petrolifera del 1973?"},
    ]

inputs = tokenizer(
[
    tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

Di seguito c'è una domanda seguita da un contesto di informazioni che completa la domanda. Rispondi alla domanda in modo appropriato.<|eot_id|><|start_header_id|>user<|end_header_id|>

Dato il seguente contesto: La crisi petrolifera del 1973 iniziò nell'ottobre 1973 quando i membri dell'Organizzazione dei Paesi esportatori di petrolio arabo (OAPEC, composta dai membri arabi dell'OPEC più Egitto e Si

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")